In [5]:
import requests
import pandas as pd

def get_basic_income_amounts(policy_id):
    # API endpoint URL
    api_url = f"https://api.policyengine.org/us/policy/{policy_id}"

    # Making a GET request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parsing the JSON response
        policy_data = response.json()

        # Extracting and renaming the basic income amounts
        basic_income_amounts = {}
        age_group_mapping = {
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": "ubi_0_5",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": "ubi_6_17",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": "ubi_18_64",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": "ubi_65_plus",
            "gov.contrib.ubi_center.basic_income.amount.person.disability": "ubi_disability"
        }

        for key, value in policy_data["result"]["policy_json"].items():
            if key in age_group_mapping:
                # Assuming the amount is the first value in the dictionary
                amount = next(iter(value.values()))
                basic_income_amounts[age_group_mapping[key]] = amount
        return basic_income_amounts
    else:
        return None

# Read the CSV file into a DataFrame
df = pd.read_csv('loss_by_rate.csv')

# Fetch and add basic income amounts for each reform ID
for i, row in df.iterrows():
    reform_id = row['api_id']
    amounts = get_basic_income_amounts(reform_id)
    if amounts:
        for column_name, amount in amounts.items():
            df.loc[i, column_name] = amount

print(df)


    flat_tax_rate      loss  api_id       ubi_0_5      ubi_6_17    ubi_18_64  \
0            0.20  0.122087   42769   3367.894501   2986.456072     1.079484   
1            0.21  0.109379   42770   3782.959548   2757.879855     3.791090   
2            0.22  0.097328   42771   3420.056606   2998.128273   126.155659   
3            0.23  0.086060   42772   3997.103949   3259.325831   441.807162   
4            0.24  0.076013   42773   3591.126570   3043.123157  1265.118286   
5            0.25  0.066969   42774   3646.868923   2738.901635  1979.095180   
6            0.26  0.059289   42775   3203.727719   2729.013652  2576.632906   
7            0.27  0.053623   42776   3144.393360   2657.716620  3164.894607   
8            0.28  0.050584   42777   3062.033136   2935.843323  3404.934731   
9            0.29  0.049405   42778   3628.349032   3076.113013  3682.353393   
10           0.30  0.049753   42767   4055.114707   3053.475804  4191.389918   
11           0.30  0.049753   42779   40

In [6]:
import plotly.express as px

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate')

# Show the plot
fig.show()